In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys

In [58]:
# Load dataset
df = pd.read_csv("dataset/resize_borvo.csv")

# Konversi ke grid dictionary
grid = {}
for _, row in df.iterrows():
    grid[(int(row["i"]), int(row["j"]))] = {"H": row["H"], "W": row["W"]}

In [59]:
# Fungsi ambil tetangga 8 arah (termasuk diagonal)
def get_all_neighbors(pos):
    i, j = pos
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0), (1, 1)
    ]
    neighbors = [(i + di, j + dj) for di, dj in directions]
    return [n for n in neighbors if n in grid]

# Fungsi hitung skor W/E
def get_score(current_pos, neighbor_pos):
    H1 = grid[current_pos]["H"]
    H2 = grid[neighbor_pos]["H"]
    S = H1 - H2

    if S > 0.5:
        return -np.inf  # Tidak aman → skip

    E = 1 + 1 * abs(S)
    W = grid[neighbor_pos]["W"]
    return W / E

In [60]:
# Fungsi Hill Climbing dengan Horizon (Lookahead Depth)
def hill_climbing_with_horizon(start, depth=5): # Saya lihat Anda pakai depth=5, jadi saya sesuaikan
    if start not in grid:
        print(f"Titik {start} tidak ditemukan dalam dataset.")
        # UBAH BAGIAN INI: Kembalikan dua list kosong
        return [], []

    current = start
    path = [current]
    visited = set([start])

    # TAMBAHKAN INI: Siapkan list untuk menyimpan data performa (nilai H)
    decision_scores = [] 

    # (Tidak ada yang berubah di fungsi dfs)
    def dfs(current_path, total_score, remaining_depth):
        current_node = current_path[-1]
        if remaining_depth == 0:
            return (total_score, current_path)

        best = (total_score, current_path)
        for neighbor in get_all_neighbors(current_node):
            if neighbor in current_path:
                continue
            score = get_score(current_node, neighbor)
            if score == -np.inf:
                continue
            result = dfs(current_path + [neighbor], total_score + score, remaining_depth - 1)
            # Logika baru Anda sepertinya memaksimalkan W/E, jadi kita cari yang terbesar
            if result[0] > best[0]:
                best = result
        return best

    def dfs(current_path, total_score, remaining_depth):
        current_node = current_path[-1]
        if remaining_depth == 0:
            return (total_score, current_path)
        best = (total_score, current_path)
        for neighbor in get_all_neighbors(current_node):
            if neighbor in current_path:
                continue
            score = get_score(current_node, neighbor)
            if score == -np.inf:
                continue
            result = dfs(current_path + [neighbor], total_score + score, remaining_depth - 1)
            if result[0] > best[0]:
                best = result
        return best

    while True:
        best_score_for_move = -np.inf
        best_path_ahead = None
        
        # Variabel untuk menyimpan skor dari langkah tunggal terbaik
        best_single_step_score = 0

        neighbors = get_all_neighbors(current)
        for neighbor in neighbors:
            if neighbor in visited:
                continue
            
            # Ini adalah skor untuk satu langkah dari `current` ke `neighbor`
            single_step_score = get_score(current, neighbor)
            if single_step_score == -np.inf:
                continue
            
            # Ini adalah skor kumulatif setelah melihat ke depan (lookahead)
            cumulative_score, result_path = dfs([current, neighbor], single_step_score, depth - 1)

            if cumulative_score > best_score_for_move:
                best_score_for_move = cumulative_score
                best_path_ahead = result_path
                # Simpan skor dari langkah pertama yang menjanjikan ini
                best_single_step_score = single_step_score

        if best_path_ahead is None or best_path_ahead[1] in visited:
            break

        current = best_path_ahead[1]
        path.append(current)
        visited.add(current)
        
        # =========================================================================
        # --- PERUBAHAN DI SINI ---
        # Simpan skor dari langkah yang baru saja diambil
        decision_scores.append(best_single_step_score)
        # =========================================================================

    # Karena ada N titik di path, maka ada N-1 langkah/skor.
    # Untuk membuat panjangnya sama agar mudah di-plot, kita tambahkan skor 0 untuk titik awal.
    final_scores_for_plotting = [0] + decision_scores

    # UBAH KEMBALIAN FUNGSI: sekarang mengembalikan skor keputusan
    return path, final_scores_for_plotting

In [61]:
# Titik uji coba
test_points = [
    (540/2, 648/2),
    (20/2, 502/2),
    (410//2, 915//2),
    (529, 357)
]

# Jalankan algoritma untuk semua titik uji
for idx, point in enumerate(test_points, 1):
    print(f"\n=== Titik {idx} - Start di {point} ===")
    path = hill_climbing_with_horizon(point)
    print(f"Jumlah langkah: {len(path)}")
    print("Jalur:")
    for p in path:
        print(p)


=== Titik 1 - Start di (270.0, 324.0) ===
Jumlah langkah: 2
Jalur:
[(270.0, 324.0), (270.0, 323.0), (269.0, 322.0), (270.0, 322.0), (270.0, 321.0), (271.0, 320.0), (270.0, 319.0), (269.0, 318.0), (268.0, 317.0), (267.0, 318.0), (266.0, 317.0), (267.0, 316.0), (267.0, 315.0), (266.0, 315.0), (266.0, 314.0), (265.0, 314.0), (265.0, 315.0), (266.0, 316.0), (265.0, 317.0), (264.0, 317.0), (263.0, 317.0), (263.0, 316.0), (263.0, 315.0), (264.0, 316.0), (265.0, 316.0), (264.0, 315.0), (263.0, 314.0), (264.0, 313.0), (264.0, 312.0), (265.0, 312.0), (266.0, 311.0), (267.0, 310.0), (268.0, 310.0), (268.0, 311.0), (267.0, 311.0), (266.0, 312.0), (267.0, 313.0), (268.0, 312.0), (269.0, 311.0), (270.0, 310.0), (270.0, 309.0), (271.0, 310.0), (270.0, 311.0), (271.0, 312.0), (271.0, 313.0), (272.0, 313.0), (273.0, 312.0), (272.0, 312.0), (272.0, 311.0), (273.0, 311.0), (274.0, 312.0), (273.0, 313.0), (272.0, 314.0), (271.0, 314.0), (270.0, 314.0), (270.0, 315.0), (269.0, 314.0), (268.0, 315.0), (26

In [62]:

# =====================================================================
# --- BAGIAN 1: KONFIGURASI SIMULASI ---
# =====================================================================
# Atur titik awal dan parameter lainnya di sini
for idx,point in enumerate(test_points,1):
    titik_awal = point
    nama_file_peta = 'europa_map/raw/resize_borvo_mensa.jpg'
    nama_file_output = f'simulasi_robot HC ke {idx}.gif'

    # =====================================================================
    # --- BAGIAN 2: PERSIAPAN DATA ---
    # =====================================================================
    print("1. Menjalankan algoritma Hill Climbing...")
    try:
        jalur, data_performa = hill_climbing_with_horizon(titik_awal, depth=5)
    except Exception as e:
        print(f"!!! TERJADI ERROR SAAT MENJALANKAN ALGORITMA: {e}")
        sys.exit() # Hentikan program jika algoritma error

    if not jalur or len(jalur) < 2:
        print(f"!!! ALGORITMA HANYA MENGHASILKAN {len(jalur)} TITIK. TIDAK CUKUP UNTUK MEMBUAT ANIMASI.")
        sys.exit()

    print(f"2. Algoritma selesai. Ditemukan jalur dengan {len(jalur)} langkah.")
    langkah_total = range(len(jalur))

    # =====================================================================
    # --- BAGIAN 3: PERSIAPAN PLOT (TIDAK AKAN DITAMPILKAN LIVE) ---
    # =====================================================================
    print("3. Menyiapkan 'panggung' untuk rendering animasi...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    fig.suptitle(f'Simulasi Robot HC ke {idx} di Europa (Start: {titik_awal})', fontsize=16)

    try:
        img = plt.imread(nama_file_peta)
        ax1.imshow(img)
    except FileNotFoundError:
        ax1.set_facecolor('black')

    jejak_robot, = ax1.plot([], [], 'b-', lw=2, alpha=0.7, label='Jejak Dilalui', zorder=5)
    posisi_robot, = ax1.plot([], [], 'ro', ms=8, label='Posisi Robot', zorder=10) 
    ax1.legend()

    ax2.set_title('Grafik Performa (Nilai Heuristik)')
    ax2.set_xlabel('Langkah')
    ax2.set_ylabel('Skor Keputusan (W/E)')
    ax2.set_xlim(0, len(jalur))
    if data_performa:
        ax2.set_ylim(min(data_performa) - 5, max(data_performa) + 5)
    ax2.grid(True)
    grafik_performa, = ax2.plot([], [], 'g-', marker='o', ms=4)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # =====================================================================
    # --- BAGIAN 4: FUNGSI UPDATE (UNTUK RENDERING) ---
    # =====================================================================
    def update(frame):
        if frame % 20 == 0:
            print(f"   ...merender frame {frame}/{len(jalur)}")
        
        path_sejauh_ini = jalur[:frame+1]
        koordinat_j = [titik[1] for titik in path_sejauh_ini]
        koordinat_i = [titik[0] for titik in path_sejauh_ini]
        
        jejak_robot.set_data(koordinat_j, koordinat_i)
        posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])
        grafik_performa.set_data(langkah_total[:frame+1], data_performa[:frame+1])
        ax1.set_title(f'Peta Penjelajahan (Langkah: {frame})')
        return jejak_robot, posisi_robot, grafik_performa,

    # =====================================================================
    # --- BAGIAN 5: RENDER DAN SIMPAN KE FILE ---
    # =====================================================================
    print("4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...")
    ani = animation.FuncAnimation(fig, update, frames=len(jalur), blit=False)

    try:
        # fps = frame per second. Mengatur seberapa cepat animasi akan diputar.
        ani.save(nama_file_output, writer='pillow', fps=20)
        
        print("\n" + "#"*60)
        print(f"###   BERHASIL! Animasi telah disimpan sebagai '{nama_file_output}'   ###")
        print("###   Silakan buka file tersebut untuk melihat hasilnya.            ###")
        print("#"*60)

    except Exception as e:
        print(f"\n!!! GAGAL MENYIMPAN ANIMASI: {e}")

1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 103 langkah.
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...
   ...merender frame 0/103
   ...merender frame 0/103


C:\Users\Rhama\AppData\Local\Temp\ipykernel_24612\3764170240.py:66: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/103
   ...merender frame 40/103
   ...merender frame 60/103
   ...merender frame 80/103
   ...merender frame 100/103

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_robot HC ke 1.gif'   ###
###   Silakan buka file tersebut untuk melihat hasilnya.            ###
############################################################
1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 79 langkah.
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...
   ...merender frame 0/79
   ...merender frame 0/79


C:\Users\Rhama\AppData\Local\Temp\ipykernel_24612\3764170240.py:66: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/79
   ...merender frame 40/79
   ...merender frame 60/79

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_robot HC ke 2.gif'   ###
###   Silakan buka file tersebut untuk melihat hasilnya.            ###
############################################################
1. Menjalankan algoritma Hill Climbing...
2. Algoritma selesai. Ditemukan jalur dengan 110 langkah.
3. Menyiapkan 'panggung' untuk rendering animasi...
4. Memulai proses rendering animasi ke file. Ini mungkin butuh waktu...
   ...merender frame 0/110
   ...merender frame 0/110


C:\Users\Rhama\AppData\Local\Temp\ipykernel_24612\3764170240.py:66: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  posisi_robot.set_data(koordinat_j[-1], koordinat_i[-1])


   ...merender frame 20/110
   ...merender frame 40/110
   ...merender frame 60/110
   ...merender frame 80/110
   ...merender frame 100/110

############################################################
###   BERHASIL! Animasi telah disimpan sebagai 'simulasi_robot HC ke 3.gif'   ###
###   Silakan buka file tersebut untuk melihat hasilnya.            ###
############################################################
1. Menjalankan algoritma Hill Climbing...
Titik (529, 357) tidak ditemukan dalam dataset.
!!! ALGORITMA HANYA MENGHASILKAN 0 TITIK. TIDAK CUKUP UNTUK MEMBUAT ANIMASI.


SystemExit: 

c:\Users\Rhama\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
